# Pulse propagation in gas-filled hollow core fiber (HCF)

## Single parameter fitting

In [ ]:
import numpy as np
from scipy.special import gamma
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft, fftshift, ifftshift, fftfreq
import warnings
warnings.filterwarnings("error")

# Defining Functions for the simulation

def getPower(amplitude):
    return np.abs(amplitude) ** 2

def getGaussianWavelengthSpectrum(wavelength, wavelength0, wavelength_FWHM):
    return np.exp(-2 * np.log(2) * ((wavelength - wavelength0) / wavelength_FWHM)**2)

def getGaussianPulseTime(time, duration):
    return np.exp(-2*np.log(2)*((time)/(duration))**2)*(1+0j)

def getSpectrumFromPulse(time,pulse_amplitude):
    dt=time[1]-time[0]
    spectrum_amplitude=fftshift(fft(pulse_amplitude))*dt # Take FFT and do shift
    return spectrum_amplitude

def analyze_pulse_characteristics(x, I, rat):
    """
    Calculates the FWHM or other width-like characteristics of a pulse.
    
    Parameters:
    - x (np.array): time or frequency array
    - I (np.array): intensity array
    - rat (float): threshold ratio (e.g., 0.5 for FWHM, 0.05 for 5% bandwidth)
    - Mult (float): optional multiplier to convert units (default: 1)
    
    Prints the pulse width in appropriate units.
    """
    max_I = np.max(I)
    I_thresh = max_I * rat

    # Find indices where intensity is above the threshold
    indices = np.where(I >= I_thresh)[0]

    if indices.size == 0:
        width_x = 0
    else:
        # Guard against out-of-bounds
        if indices[0] == 0 or indices[-1] == len(x) - 1:
            print("Warning: Threshold too close to boundary. Skipping interpolation.")
            xa1 = x[indices[0]]
            xa2 = x[indices[-1]]
        else:
            # Linear interpolation around threshold crossings
            x1, x2 = x[indices[0] - 1], x[indices[0]]
            f1, f2 = I[indices[0] - 1], I[indices[0]]
            xa1 = ((I_thresh - f1) * x2 + (f2 - I_thresh) * x1) / (f2 - f1)

            x3, x4 = x[indices[-1]], x[indices[-1] + 1]
            f3, f4 = I[indices[-1]], I[indices[-1] + 1]
            xa2 = ((I_thresh - f4) * x3 + (f3 - I_thresh) * x4) / (f3 - f4)

        width_x = xa2 - xa1

    return width_x

def calculate_refractive_index(pressure_bar):
    """
    Calculate the refractive index of CO2 at a given wavelength and pressure.
    
    Assumes linear pressure scaling from known value at 1 atm.
    
    Parameters:
    - wavelength_nm: Wavelength in nanometers
    - pressure_bar: Pressure in bar
    
    Returns:
    - Refractive index of CO2 at given pressure
    """
    # Reference data: n_CO2 at 1030 nm and 1 atm (~293 K)
    n_1atm = 1.00045  # Approximate value at 1030 nm from literature
    atm_pressure = 1.01325  # 1 atm in bar

    delta_n = (n_1atm - 1) * (pressure_bar / atm_pressure)
    n = 1 + delta_n
    return n

def calculate_n2(pressure_bar, n2_atm=3.0e-19):
    """
    Calculate the nonlinear refractive index n2 of CO2 at given pressure.

    Parameters:
    - pressure_bar: Pressure in bar
    - n2_atm: n2 of CO2 at 1 atm in cm^2/W (default: 3e-19)

    Returns:
    - n2 in m^2/W
    """
    atm_to_bar = 1.01325
    n2_cm2_W = n2_atm * (pressure_bar / atm_to_bar)
    return n2_cm2_W * 1e-4  # convert to m^2/W

def calculate_gamma(n2_atm, pressure_bar, wavelength, core_diameter):
    """
    Calculate gamma from physical parameters and pressure-scaled n2.

    Parameters:
    - n2_atm: Nonlinear index at 1 atm (in cm^2/W)
    - pressure_bar: Gas pressure in bar
    - wavelength: Wavelength in meters
    - core_diameter: Diameter of fiber core in meters

    Returns:
    - gamma in W^-1·m^-1
    """
    n2 = calculate_n2(pressure_bar, n2_atm)
    core_radius = core_diameter / 2
    A_eff = np.pi * core_radius**2
    gamma = (2 * np.pi * n2) / (wavelength * A_eff)
    return gamma

def mittag_leffler_series(alpha, z, K=100):
    result = 0
    for k in range(K):
        result += z**k / gamma(alpha * k + 1)
    return result

def mittag_leffler_array(alpha, arg_array):
    return np.array([mittag_leffler_series(alpha, element) for element in arg_array], dtype=np.complex128)

# Defining parameters for the simulation
speedoflight_nmpfs = 300                                                         
wavelength0_nm = 1030
wavelength_FWHM_nm = 30
frequency_FWHM = wavelength_FWHM_nm * speedoflight_nmpfs / wavelength0_nm**2
duration_FWHM_fs = 0.44 /  frequency_FWHM
wavelength_frontend_nm = np.arange(730, 1331, 1)
spectrum_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_frontend_nm, wavelength0_nm, wavelength_FWHM_nm)
I_frontend = getPower(spectrum_amplitude_frontend)
speedoflight_mps = 3e8
wavelength0_m = wavelength0_nm * 1e-9
wavelength_frontend_m = wavelength_frontend_nm * 1e-9
wavelength_FWHM_m = wavelength_FWHM_nm * 1e-9
frequency0=speedoflight_mps/wavelength0_m                       
omega0=2*np.pi*frequency0                                   
core_diameter = 250e-6      
pressure = 0.5        
n2_atm = 3.0e-19
true_alpha = 1.0            

# Compute parameters
refractive_index = calculate_refractive_index(pressure)
beta0 = refractive_index * (omega0 / speedoflight_mps)
#gamma_calculated = calculate_gamma(n2_atm, pressure, wavelength0_m, core_diameter)
gamma_calculated = 20

# Time, frequency and wavelength grid
N=2**10
Time_window = 50 * duration_FWHM_fs * 1e-15                                        
t = np.linspace(-Time_window/2,Time_window/2,N)                                                                                  
dt = abs(t[1] - t[0])                                   
f = fftshift(fftfreq(N,d=dt))
f_rel = f + frequency0
wavelength_rel = speedoflight_mps / f_rel
sort_idx = np.argsort(wavelength_rel)
wavelength_rel = wavelength_rel[sort_idx]

# Compute inverse Jacobian: dλ/df
inv_jacobian = (wavelength_rel**2) / (speedoflight_mps)

# Prppagation distance
z = 1

# Interpolate frontend spectrum onto simulated wavelength grid (wavelength_rel)
interp_func = interp1d(wavelength_frontend_m, I_frontend, kind='cubic', fill_value=0, bounds_error=False)
I_frontend_simgrid = interp_func(wavelength_rel)

# Normalize
I_frontend_simgrid /= np.max(I_frontend_simgrid)
#I_measured_simgrid = np.clip(I_measured_simgrid, 0, None)

initial_pulse = getGaussianPulseTime(t, duration_FWHM_fs * 1e-15)

# Reference measured spectrum (can be simulated with a known alpha)
arg_true = - gamma_calculated * beta0**(1-true_alpha) * np.exp(-1j * np.pi * true_alpha / 2) * getPower(initial_pulse) * z ** true_alpha
pulse_true = initial_pulse * mittag_leffler_array(true_alpha, arg_true)
pulse_true_fft = getSpectrumFromPulse(t,pulse_true)
spec_ref_frequency = getPower(pulse_true_fft)
spec_ref_frequency /= np.max(spec_ref_frequency)
spec_ref = spec_ref_frequency * inv_jacobian
spec_ref /= np.max(spec_ref)

# --- Loss function: spectral L2 distance ---
def spectral_loss(alpha):
    if not (0.9 < alpha < 1.0):
        return np.inf
    arg_model = - gamma_calculated * beta0**(1-alpha) * np.exp(-1j * np.pi * true_alpha / 2) * getPower(initial_pulse) * z ** alpha
    pulse_model = initial_pulse * mittag_leffler_array(alpha, arg_model)
    pulse_model_fft = getSpectrumFromPulse(t,pulse_model)
    spec_model_frequency = getPower(pulse_model_fft)
    spec_model_frequency /= np.max(spec_model_frequency)
    spec_model = spec_model_frequency * inv_jacobian
    spec_model /= np.max(spec_model)
    return np.sum((spec_model - spec_ref)**2)

# --- another loss function ---
def spectral_loss2(alpha):
    if not (0.9 < alpha < 1.0):
        return np.inf
    arg_model = - gamma_calculated * beta0**(1-alpha) * np.exp(-1j * np.pi * true_alpha / 2) * getPower(initial_pulse) * z ** alpha
    pulse_model = initial_pulse * mittag_leffler_array(alpha, arg_model)
    pulse_model_fft = getSpectrumFromPulse(t,pulse_model)
    spec_model_frequency = getPower(pulse_model_fft)
    spec_model_frequency /= np.max(spec_model_frequency)
    spec_model = spec_model_frequency * inv_jacobian
    spec_model /= np.max(spec_model)
    return np.sum((spec_model - np.mean(spec_ref))**2)

# --- Estimate alpha ---
res = minimize_scalar(spectral_loss, bounds=(0.9, 1.0), method='bounded')
fitted_alpha = res.x
print(f"Fitted alpha: {fitted_alpha}")

# Fitted spectrum with estimated alpha
arg_fit = - gamma_calculated * beta0**(1-fitted_alpha) * np.exp(-1j * np.pi * true_alpha / 2) * getPower(initial_pulse) * z ** fitted_alpha
pulse_fit = initial_pulse * mittag_leffler_array(fitted_alpha, arg_fit)
pulse_fit_fft = getSpectrumFromPulse(t, pulse_fit)
spec_fit_frequency = getPower(pulse_fit_fft)
spec_fit_frequency /= np.max(spec_fit_frequency)
spec_fit = spec_fit_frequency * inv_jacobian
spec_fit /= np.max(spec_fit)

# Frontend spectrum
spec_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_rel, wavelength0_m, wavelength_FWHM_m)
spec_frontend = getPower(spec_amplitude_frontend)

# Residual sum of squares
ss_res = spectral_loss(fitted_alpha)
# Total sum of squares
ss_tot = spectral_loss2(fitted_alpha)
# R^2 value
r_squared = 1 - (ss_res / ss_tot)
print(f"R²: {r_squared}")

# --- Plot comparison ---
plt.figure(figsize=(10,5))
plt.plot(wavelength_rel * 1e9, spec_fit, label=f"Analitical α={fitted_alpha}", linestyle='--')
plt.plot(wavelength_rel * 1e9, spec_ref, label=f"Reference α={true_alpha}")
plt.plot(wavelength_rel * 1e9, spec_frontend, label="Frontend")
plt.axis([600,2000,0,1])
plt.xlabel("Wavelength [nm]")
plt.ylabel("Normalized power spectral density [a.u.]")
plt.title(f"Spectral Matching for α Fitting, R²: {r_squared}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Two parameter estimation with 5% bandwidth

In [ ]:
import numpy as np
from scipy.special import gamma
from scipy.optimize import minimize
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft, fftshift, ifftshift, fftfreq
import warnings
warnings.filterwarnings("error")

# Defining Functions for the simulation

def getPower(amplitude):
    return np.abs(amplitude) ** 2

def getGaussianWavelengthSpectrum(wavelength, wavelength0, wavelength_FWHM):
    return np.exp(-2 * np.log(2) * ((wavelength - wavelength0) / wavelength_FWHM)**2)

def getGaussianPulseTime(time, duration):
    return np.exp(-2*np.log(2)*((time)/(duration))**2)*(1+0j)

def getSpectrumFromPulse(time,pulse_amplitude):
    dt=time[1]-time[0]
    spectrum_amplitude=fftshift(fft(pulse_amplitude))*dt # Take FFT and do shift
    return spectrum_amplitude

def analyze_pulse_characteristics(x, I, rat):
    """
    Calculates the FWHM or other width-like characteristics of a pulse.
    
    Parameters:
    - x (np.array): time or frequency array
    - I (np.array): intensity array
    - rat (float): threshold ratio (e.g., 0.5 for FWHM, 0.05 for 5% bandwidth)
    - Mult (float): optional multiplier to convert units (default: 1)
    
    Prints the pulse width in appropriate units.
    """
    max_I = np.max(I)
    I_thresh = max_I * rat

    # Find indices where intensity is above the threshold
    indices = np.where(I >= I_thresh)[0]

    if indices.size == 0:
        width_x = 0
    else:
        # Guard against out-of-bounds
        if indices[0] == 0 or indices[-1] == len(x) - 1:
            print("Warning: Threshold too close to boundary. Skipping interpolation.")
            xa1 = x[indices[0]]
            xa2 = x[indices[-1]]
        else:
            # Linear interpolation around threshold crossings
            x1, x2 = x[indices[0] - 1], x[indices[0]]
            f1, f2 = I[indices[0] - 1], I[indices[0]]
            xa1 = ((I_thresh - f1) * x2 + (f2 - I_thresh) * x1) / (f2 - f1)

            x3, x4 = x[indices[-1]], x[indices[-1] + 1]
            f3, f4 = I[indices[-1]], I[indices[-1] + 1]
            xa2 = ((I_thresh - f4) * x3 + (f3 - I_thresh) * x4) / (f3 - f4)

        width_x = xa2 - xa1

    return width_x

def calculate_refractive_index(pressure_bar):
    """
    Calculate the refractive index of CO2 at a given wavelength and pressure.
    
    Assumes linear pressure scaling from known value at 1 atm.
    
    Parameters:
    - wavelength_nm: Wavelength in nanometers
    - pressure_bar: Pressure in bar
    
    Returns:
    - Refractive index of CO2 at given pressure
    """
    # Reference data: n_CO2 at 1030 nm and 1 atm (~293 K)
    n_1atm = 1.00045  # Approximate value at 1030 nm from literature
    atm_pressure = 1.01325  # 1 atm in bar

    delta_n = (n_1atm - 1) * (pressure_bar / atm_pressure)
    n = 1 + delta_n
    return n

def mittag_leffler_series(alpha, z, K=100):
    result = 0
    for k in range(K):
        result += z**k / gamma(alpha * k + 1)
    return result

def mittag_leffler_array(alpha, arg_array):
    return np.array([mittag_leffler_series(alpha, element) for element in arg_array], dtype=np.complex128)

# Defining parameters for the simulation
speedoflight_nmpfs = 300                                                         
wavelength0_nm = 1030
wavelength_FWHM_nm = 30
frequency_FWHM = wavelength_FWHM_nm * speedoflight_nmpfs / wavelength0_nm**2
duration_FWHM_fs = 0.44 /  frequency_FWHM
wavelength_frontend_nm = np.arange(730, 1331, 1)
spectrum_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_frontend_nm, wavelength0_nm, wavelength_FWHM_nm)
I_frontend = getPower(spectrum_amplitude_frontend)
speedoflight_mps = 3e8
wavelength0_m = wavelength0_nm * 1e-9
wavelength_frontend_m = wavelength_frontend_nm * 1e-9
wavelength_FWHM_m = wavelength_FWHM_nm * 1e-9
frequency0=speedoflight_mps/wavelength0_m                       
omega0=2*np.pi*frequency0                                         
pressure = 0.5        
n2_atm = 3.0e-19
true_alpha = 0.95            

# Compute parameters
refractive_index = calculate_refractive_index(pressure)
beta0 = refractive_index * (omega0 / speedoflight_mps)

# Time, frequency and wavelength grid
N=2**10
Time_window = 50 * duration_FWHM_fs * 1e-15                                        
t = np.linspace(-Time_window/2,Time_window/2,N)                                                                                  
dt = abs(t[1] - t[0])                                   
f = fftshift(fftfreq(N,d=dt))
f_rel = f + frequency0
wavelength_rel = speedoflight_mps / f_rel
sort_idx = np.argsort(wavelength_rel)
wavelength_rel = wavelength_rel[sort_idx]

# Compute inverse Jacobian: dλ/df
inv_jacobian = (wavelength_rel**2) / (speedoflight_mps)

# Prppagation distance
z = 1

# Interpolate frontend spectrum onto simulated wavelength grid (wavelength_rel)
interp_func = interp1d(wavelength_frontend_m, I_frontend, kind='cubic', fill_value=0, bounds_error=False)
I_frontend_simgrid = interp_func(wavelength_rel)

# Normalize
I_frontend_simgrid /= np.max(I_frontend_simgrid)
#I_frontend_simgrid = np.clip(I_frontend_simgrid, 0, None)

initial_pulse = getGaussianPulseTime(t, duration_FWHM_fs * 1e-15)

def simulate_spectrum(alpha, gamma_value):
    arg_model = - gamma_value * beta0**(1-alpha) * np.exp(-1j * np.pi * alpha / 2) * getPower(initial_pulse) * z ** alpha
    pulse_model = initial_pulse * mittag_leffler_array(alpha, arg_model)
    pulse_model_fft = getSpectrumFromPulse(t, pulse_model)
    spec_model_freq = getPower(pulse_model_fft)
    spec_model_freq /= np.max(spec_model_freq)
    spec_model = spec_model_freq * inv_jacobian
    spec_model /= np.max(spec_model)
    return spec_model

bw_measured = 118.6 * 1e-9 # meter

def loss_bandwidth(params):
    alpha, gamma_val = params
    if not (0.9 < alpha < 1.0) or not (1 < gamma_val < 20):
        return np.inf
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("error")  # Convert warnings to exceptions
            spec_model = simulate_spectrum(alpha, gamma_val)
            bw_sim = analyze_pulse_characteristics(wavelength_rel, spec_model, 0.05)
            if np.isnan(bw_sim) or np.isinf(bw_sim):
                return np.inf
            return (bw_sim - bw_measured) ** 2
    except (FloatingPointError, RuntimeWarning, ValueError) as e:
        return np.inf

initial_guess = [0.95, 6] # alpha and gamma respectively
bounds = [(0.90, 1.0), (1, 20)] # alpha and gamma respectively

result = minimize(loss_bandwidth, initial_guess, bounds=bounds, method='Powell')

if result.success:
    alpha_est, gamma_est = result.x
    spec_model = simulate_spectrum(alpha_est, gamma_est)
    bw_sim = analyze_pulse_characteristics(wavelength_rel, spec_model, 0.05)
    loss = (bw_sim - bw_measured) ** 2
    print(f"✅ Optimization succeeded")
    print(f"Estimated alpha: {alpha_est}")
    print(f"Estimated gamma: {gamma_est}")
    print(f"Loss Bandwidth: {loss}")
else:
    print("❌ Optimization failed:", result.message)

🎯 Why Use a Loss Landscape?

| Benefit                                | What It Tells You                                                                         |
| -------------------------------------- | ----------------------------------------------------------------------------------------- |
| 🔍 **Global minimum location**         | Where the best-fit parameters lie                                                         |
| 📉 **Sensitivity**                     | Which parameters affect the loss more — steep vs. flat regions                            |
| 🧠 **Parameter coupling / degeneracy** | Whether two parameters can compensate for each other (e.g., α↑, γ↓ still gives same loss) |
| 🛠️ **Optimizer diagnostics**          | Whether your optimizer might get stuck in local minima or flat regions                    |
| 💡 **Confidence regions**              | Regions where the loss is low → potentially acceptable solutions                          |


🧠 How to Interpret a Loss Landscape

| Shape or Feature            | Meaning                                                                                 |
| --------------------------- | --------------------------------------------------------------------------------------- |
| 🎯 **Sharp global minimum** | Well-constrained parameters, optimization is easier                                     |
| ⛰️ **Multiple minima**      | Optimization might get stuck — choose a global method like `differential_evolution`     |
| 🏔️ **Valley-shaped**       | One parameter tightly constrained, one loosely (you may have a degenerate solution set) |
| 🪨 **Flat regions**         | Poor parameter identifiability — small changes don't affect the loss                    |
| 🧩 **Ridges or couplings**  | One parameter compensates for another — they're entangled (e.g., α vs. γ)               |


## Two parameter fitting with spectrum

In [ ]:
import numpy as np
from scipy.special import gamma
from scipy.optimize import minimize
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.widgets import Slider
from scipy.fftpack import fft, ifft, fftshift, ifftshift, fftfreq
import warnings
warnings.filterwarnings("error")

# Defining Functions for the simulation

def getPower(amplitude):
    return np.abs(amplitude) ** 2

def getGaussianWavelengthSpectrum(wavelength, wavelength0, wavelength_FWHM):
    return np.exp(-2 * np.log(2) * ((wavelength - wavelength0) / wavelength_FWHM)**2)

def getGaussianPulseTime(time, duration):
    return np.exp(-2*np.log(2)*((time)/(duration))**2)*(1+0j)

def getSpectrumFromPulse(time,pulse_amplitude):
    dt=time[1]-time[0]
    spectrum_amplitude=fftshift(fft(pulse_amplitude))*dt # Take FFT and do shift
    return spectrum_amplitude

def calculate_refractive_index(pressure_bar):
    """
    Calculate the refractive index of CO2 at a given wavelength and pressure.
    
    Assumes linear pressure scaling from known value at 1 atm.
    
    Parameters:
    - wavelength_nm: Wavelength in nanometers
    - pressure_bar: Pressure in bar
    
    Returns:
    - Refractive index of CO2 at given pressure
    """
    # Reference data: n_CO2 at 1030 nm and 1 atm (~293 K)
    n_1atm = 1.00045  # Approximate value at 1030 nm from literature
    atm_pressure = 1.01325  # 1 atm in bar

    delta_n = (n_1atm - 1) * (pressure_bar / atm_pressure)
    n = 1 + delta_n
    return n

def mittag_leffler_series(alpha, z, K=150):
    result = 0
    for k in range(K):
        result += z**k / gamma(alpha * k + 1)
    return result

def mittag_leffler_array(alpha, arg_array):
    return np.array([mittag_leffler_series(alpha, element) for element in arg_array], dtype=np.complex128)

# Defining parameters for the simulation
speedoflight_nmpfs = 300                                                         
wavelength0_nm = 1030
wavelength_FWHM_nm = 30
frequency_FWHM = wavelength_FWHM_nm * speedoflight_nmpfs / wavelength0_nm**2
duration_FWHM_fs = 0.44 /  frequency_FWHM
wavelength_frontend_nm = np.arange(730, 1331, 1)
spectrum_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_frontend_nm, wavelength0_nm, wavelength_FWHM_nm)
I_frontend = getPower(spectrum_amplitude_frontend)
speedoflight_mps = 3e8
wavelength0_m = wavelength0_nm * 1e-9
wavelength_frontend_m = wavelength_frontend_nm * 1e-9
wavelength_FWHM_m = wavelength_FWHM_nm * 1e-9
frequency0=speedoflight_mps/wavelength0_m                       
omega0=2*np.pi*frequency0                                        
pressure = 0.5        

# Compute parameters
refractive_index = calculate_refractive_index(pressure)
beta0 = refractive_index * (omega0 / speedoflight_mps)

# Time, frequency and wavelength grid
N=2**10
Time_window = 50 * duration_FWHM_fs * 1e-15                                        
t = np.linspace(-Time_window/2,Time_window/2,N)                                                                                  
dt = abs(t[1] - t[0])                                   
f = fftshift(fftfreq(N,d=dt))
f_rel = f + frequency0
wavelength_rel = speedoflight_mps / f_rel
sort_idx = np.argsort(wavelength_rel)
wavelength_rel = wavelength_rel[sort_idx]

# Compute inverse Jacobian: dλ/df
inv_jacobian = (wavelength_rel**2) / (speedoflight_mps)

# Prppagation distance
z = 1

# Interpolate frontend spectrum onto simulated wavelength grid (wavelength_rel)
interp_func = interp1d(wavelength_frontend_m, I_frontend, kind='cubic', fill_value=0, bounds_error=False)
I_frontend_simgrid = interp_func(wavelength_rel)

# Normalize
I_frontend_simgrid /= np.max(I_frontend_simgrid)
#I_frontend_simgrid = np.clip(I_frontend_simgrid, 0, None)

# Load the CSV file, skip the header row
data = np.loadtxt("Meas_175.csv", delimiter=";", skiprows=1)
# Split into separate arrays
wavelength_spec_ref_nm = data[:, 0]
wavelength_spec_ref_m = wavelength_spec_ref_nm * 1e-9
I_spec_ref = data[:, 2]

# Interpolate measured spectrum onto simulated wavelength grid (wavelength_rel)
interp_func = interp1d(wavelength_spec_ref_m, I_spec_ref, kind='cubic', fill_value=0, bounds_error=False)
I_spec_ref_simgrid = interp_func(wavelength_rel)

# Normalize
I_spec_ref_simgrid /= np.max(I_spec_ref_simgrid)

"""
# --- Plot comparison ---
plt.figure(figsize=(10,5))
plt.plot(wavelength_rel * 1e9, I_spec_ref_simgrid, label=f"Reference")
plt.plot(wavelength_rel * 1e9, I_frontend_simgrid, label="Frontend")
plt.axis([800,1200,0,1])
plt.xlabel("Wavelength [nm]")
plt.ylabel("Normalized power spectral density [a.u.]")
plt.title(f"Spectrums")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
"""

initial_pulse = getGaussianPulseTime(t, 1000 * 1e-15)

def simulate_spectrum(alpha, gamma_value):
    arg_model = - gamma_value * beta0**(1-alpha) * np.exp(-1j * np.pi * alpha / 2) * getPower(initial_pulse) * z ** alpha
    pulse_model = initial_pulse * mittag_leffler_array(alpha, arg_model)
    pulse_model_fft = getSpectrumFromPulse(t, pulse_model)
    spec_model_freq = getPower(pulse_model_fft)
    spec_model_freq /= np.max(spec_model_freq)
    spec_model = spec_model_freq * inv_jacobian
    spec_model /= np.max(spec_model)
    return spec_model

def spectral_loss(params):
    alpha, gamma_val = params
    if not (0.1 < alpha <= 1.0) or not (1e-5 <= gamma_val < 1e2):
        #print(params)
        return np.inf
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("error")  # Convert warnings to exceptions
            spec_model = simulate_spectrum(alpha, gamma_val)
            spec_ref = I_spec_ref_simgrid
            return np.sum((spec_model - spec_ref)**2)
    except (FloatingPointError, RuntimeWarning, ValueError) as e:
        #print(params)
        return np.inf

def spectral_loss2(params):
    alpha, gamma_val = params
    if not (0.1 < alpha <= 1.0) or not (1e-5 <= gamma_val < 1e2):
        #print(params)
        return np.inf
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("error")  # Convert warnings to exceptions
            spec_model = simulate_spectrum(alpha, gamma_val)
            spec_ref = I_spec_ref_simgrid
            return np.sum((spec_model - np.mean(spec_ref))**2)
    except (FloatingPointError, RuntimeWarning, ValueError) as e:
        #print(params)
        return np.inf

# Create figure and axes
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.55)  # Extra space for sliders

# Initial parameter values
alpha_init = 1
gamma_init = 1e-3
z_init = 1.0            # in meters
amp_init = 1.0          # amplitude scale
fwhm_init = 500         # in femtoseconds

# --- Simulation Function ---
def run_sim(alpha, gamma, z, amp_scale, fwhm_fs):
    try:
        pulse = getGaussianPulseTime(t, fwhm_fs * 1e-15) * np.sqrt(amp_scale)
        power = getPower(pulse)
        arg_model = - gamma * beta0**(1 - alpha) * np.exp(-1j * np.pi * alpha / 2) * power * z ** alpha
        pulse_model = pulse * mittag_leffler_array(alpha, arg_model)
        spectrum = getSpectrumFromPulse(t, pulse_model)
        I_sim = getPower(spectrum)
        if np.max(I_sim) == 0 or np.isnan(np.max(I_sim)):
            return np.zeros_like(I_sim)
        I_sim /= np.max(I_sim)
        I_sim *= inv_jacobian
        I_sim /= np.max(I_sim)
        return I_sim
    except Exception as e:
        print(f"Simulation error: {e}")
        return np.zeros_like(inv_jacobian)

# --- Initial plot ---
spectrum = run_sim(alpha_init, gamma_init, z_init, amp_init, fwhm_init)
l, = ax.plot(wavelength_rel * 1e9, spectrum, label='Simulated')
l2, = ax.plot(wavelength_rel * 1e9, I_spec_ref_simgrid, '--', label='Measured')

# R² text placeholder
r2_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=10, verticalalignment='top')

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Normalized Intensity")
ax.set_title("Fractional Propagation Simulation")
ax.grid(True)
ax.set_ylim(0, 1)
ax.set_xlim(900, 1200)
ax.legend()

# --- Slider axes ---
ax_alpha = plt.axes([0.25, 0.42, 0.65, 0.03])
ax_log_gamma = plt.axes([0.25, 0.35, 0.65, 0.03])
ax_z = plt.axes([0.25, 0.28, 0.65, 0.03])
ax_amp = plt.axes([0.25, 0.21, 0.65, 0.03])
ax_fwhm = plt.axes([0.25, 0.14, 0.65, 0.03])

# --- Sliders ---
slider_alpha = Slider(ax_alpha, 'Alpha (α)', 0.98, 1.0, valinit=alpha_init, valstep=0.0001)
slider_log_gamma = Slider(ax_log_gamma, 'log₁₀(γ)', -10, 2, valinit=np.log10(gamma_init), valstep=0.01)
slider_z = Slider(ax_z, 'z [m]', 0.0, 10.0, valinit=z_init, valstep=0.1)
slider_amp = Slider(ax_amp, 'Amplitude Scale', 0.1, 10.0, valinit=amp_init, valstep=0.01)
slider_fwhm = Slider(ax_fwhm, 'Pulse FWHM [fs]', 100, 1000.0, valinit=fwhm_init, valstep=1.0)

# --- Update function ---
def update(val):
    alpha = slider_alpha.val
    gamma = 10 ** slider_log_gamma.val
    z = slider_z.val
    amp = slider_amp.val
    fwhm_fs = slider_fwhm.val

    # Run simulation
    new_spec = run_sim(alpha, gamma, z, amp, fwhm_fs)
    l.set_ydata(new_spec)

    # Compute R²
    y_true = I_spec_ref_simgrid
    y_pred = new_spec
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    # Update title and R² text
    ax.set_title(f"α = {alpha:.4f}, γ = {gamma:.2e}, z = {z:.2f} m, Amp = {amp:.2f}, FWHM = {fwhm_fs:.1f} fs")
    r2_text.set_text(f"R² = {r_squared:.5f}")
    fig.canvas.draw_idle()

# Connect sliders
slider_alpha.on_changed(update)
slider_log_gamma.on_changed(update)
slider_z.on_changed(update)
slider_amp.on_changed(update)
slider_fwhm.on_changed(update)

# Show plot
plt.show()
# === END SLIDER UI WITH R² ===

"""
initial_guess = [0.5, 1e-3] # alpha and gamma respectively
bounds = [(0.1, 1.0), (1e-5, 1e2)] # alpha and gamma respectively

result = minimize(spectral_loss, x0=initial_guess, bounds=bounds, method='L-BFGS-B')

if result.success:
    alpha_fit, gamma_fit = result.x
    # Residual sum of squares
    ss_res = spectral_loss(result.x)
    # Total sum of squares
    ss_tot = spectral_loss2(result.x)
    # R^2 value
    r_squared = 1 - (ss_res / ss_tot)
    print(f"✅ Fitting succeeded")
    print(f"Fitted alpha: {alpha_fit}")
    print(f"Fitted gamma: {gamma_fit}")
    print(f"R²: {r_squared}")
else:
    print("❌ Fitting failed:", result.message)
"""


'\ninitial_guess = [0.5, 1e-3] # alpha and gamma respectively\nbounds = [(0.1, 1.0), (1e-5, 1e2)] # alpha and gamma respectively\n\nresult = minimize(spectral_loss, x0=initial_guess, bounds=bounds, method=\'L-BFGS-B\')\n\nif result.success:\n    alpha_fit, gamma_fit = result.x\n    # Residual sum of squares\n    ss_res = spectral_loss(result.x)\n    # Total sum of squares\n    ss_tot = spectral_loss2(result.x)\n    # R^2 value\n    r_squared = 1 - (ss_res / ss_tot)\n    print(f"✅ Fitting succeeded")\n    print(f"Fitted alpha: {alpha_fit}")\n    print(f"Fitted gamma: {gamma_fit}")\n    print(f"R²: {r_squared}")\nelse:\n    print("❌ Fitting failed:", result.message)\n'